In [1]:
!nvidia-smi

Fri Apr 29 21:32:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 441.45       Driver Version: 441.45       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce MX350      WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   51C    P8    N/A /  N/A |     68MiB /  2048MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras import regularizers

In [3]:
maxlen=100 #  在100个词后截断评论
max_words=50000 # 考虑前10000个出现的词 发现有134658个word_index，故调整为50000

In [4]:
data=np.load("./data_50000_100.npy")
labels=np.load("./labels.npy")
# Embedding_matrix=np.load("./Embedding/Embedding_matrix_100.npy")

In [5]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
def build_model_cnn_GRU():
    Input_=Input(shape=(maxlen,),name='Input_data')
    Embed=layers.Embedding(max_words,128)(Input_)
    Conv_1=layers.Conv1D(128,3,activation='relu',input_shape=(maxlen,128))(Embed)
    Max_p=layers.MaxPooling1D(3)(Conv_1)
    Conv_2=layers.Conv1D(128,3,activation='relu')(Max_p)
    GRU_1=layers.Bidirectional(layers.GRU(64,recurrent_dropout=0.5,activation='relu'))(Conv_2)
    GRU_1=layers.Dropout(0.5)(GRU_1)
    Dense_1=layers.Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.001))(GRU_1)
    Dense_2=layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001))(Dense_1)
    Output_=layers.Dense(1,activation='sigmoid')(Dense_2)
    model=Model(Input_,Output_)
    model.summary()
    return model

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint( './models/CNN_GRU_{epoch:02d}.h5',
                                   verbose=1, mode='auto',monitor='val_acc',save_best_only=True,period=1)

In [7]:
model=build_model_cnn_GRU()
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['acc'])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_data (InputLayer)      [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 128)          6400000   
_________________________________________________________________
conv1d (Conv1D)              (None, 98, 128)           49280     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 32, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 30, 128)           49280     
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               74496     
_________________________________________________________________
dropout (Dropout)            (None, 128)              

In [ ]:
history=model.fit(data,
          labels,
          epochs=20,
          batch_size=128,
          validation_split=0.2,
            callbacks=[checkpointer])

Epoch 1/20
2324/6250 [==========>...................] - ETA: 26:13 - loss: 0.6371 - acc: 0.6395